# HW1          by Trisha Mandal 

In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
import re
from bs4 import BeautifulSoup
from textblob import TextBlob
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/trishamandal/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
! pip install bs4 # in case you don't have it installed

# Dataset: https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Jewelry_v1_00.tsv.gz

In [3]:
pip install contractions

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install textblob

Note: you may need to restart the kernel to use updated packages.


## Read Data

In [5]:
# reading from the given dataset by using pd.read_table
df = pd.read_table('amazon_reviews_us_Jewelry_v1_00.tsv', error_bad_lines=False,warn_bad_lines=False)

## Keep Reviews and Ratings

In [6]:
# I have saved only the star_ratings and review_body in a dataframe.
data = pd.concat([df['star_rating'], df['review_body']], axis=1)

 ## We select 20000 reviews randomly from each rating class.



In [7]:
# I have selected 20000 ratings from each rating category to have balanced data by using .sample method on the 
# dataframe and then combining 20000 reviews of each rating into 1 dataframe 
data = pd.concat([df['star_rating'], df['review_body']], axis=1)
data = data.dropna()
#data.drop(data[data['review_body'].str.split().str.len() < 10].index, inplace=True)
data['star_rating']= [int(i) for i in data['star_rating']]
rating1 = data[data['star_rating'] == 1].sample(n=20000, random_state = 2)
rating2 = data[data['star_rating'] == 2].sample(n=20000, random_state = 2)
rating3 = data[data['star_rating'] == 3].sample(n=20000, random_state = 2)
rating4 = data[data['star_rating'] == 4].sample(n=20000, random_state = 2)
rating5 = data[data['star_rating'] == 5].sample(n=20000, random_state = 2)
combined = pd.concat([rating1, rating2, rating3, rating4, rating5])

# Data Cleaning



In [8]:
# keeping a copy of the combined dataframe after sampling
combinedcopy = combined.copy()

In [9]:
# converting all the reviews into string
combined['review_body']= [str(i) for i in combined['review_body']]

In [10]:
#Average length of words before data cleaning by using mean on length of string
avg = combined['review_body'].apply(lambda x: len(x))
avgwordlength1 = avg.mean()

In [11]:
# data cleaning on sampled data
# step 1: changing all words to lower case by using str.lower()
# step 2: performing contractions on the reviews by using contractions library
# step 3: Removing HTML by using BeatifulSoup library
# step 4: Removing URLS by using regex
# step 5: Removing non-alphanumeric characters by using regex
# step 6: Stripping extra space
# step 7: Replacing double spaces with single spaces
import contractions
combined['review_body'] = combined['review_body'].str.lower()
combined['review_body'] = combined['review_body'].apply(lambda x: contractions.fix(x))
combined['review_body'] = [BeautifulSoup(text).get_text() for text in combined['review_body']]
combined['review_body'] = combined['review_body'].apply(lambda text: re.sub(r'www.\S+.com', '', text))
combined['review_body'] = combined['review_body'].apply(lambda text: re.sub(r'https?:\S+', '', text)) 
combined['review_body'] = combined['review_body'].apply(lambda x: re.sub('\W+', ' ', x))
combined["review_body"] = combined["review_body"].apply(lambda x: re.sub(r"\d+", ' ', x))
combined["review_body"] = combined["review_body"].apply(lambda x: x.strip()) 
combined["review_body"] = combined["review_body"].apply(lambda x: x.replace("  ", " "))
#combined["review_body"] = combined["review_body"].apply(lambda x: TextBlob(x).correct())
combined

,star_rating,review_body
948126,1,i was disappointed with the quality and the si...
32115,1,cheap and ugly
752868,1,do not waste your money depicts silver studs b...
1155702,1,if you are toying with the idea of buying this...
908775,1,way too long
...,...,...
1330384,5,this charm fits great on my pandora bracelet a...
1603227,5,i am so glad that i purchased this bracelet i ...
1025288,5,bought this as a gift for a friend he absolute...
1522848,5,this is quite a find especially not because of...


In [12]:
#Average length of words after data cleaning by using mean on length of string
avg = combined['review_body'].apply(lambda x: len(x))
avgwordlength2 = avg.mean()
print("Average Review Length before and after data cleaning: {}, {} ".format(avgwordlength1, avgwordlength2))

Average Review Length before and after data cleaning: 189.88698, 183.31113 


# Pre-processing

## remove the stop words 

In [13]:
#removing stop words by using stopwords from nltk library
from nltk.corpus import stopwords
stop = stopwords.words('english')
combined['review_body'] = combined['review_body'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [14]:
#Average length of words after removing stop words by using mean on length of string
avg = combined['review_body'].apply(lambda x: len(x))
avgwordlength3 = avg.mean()

## perform lemmatization  

In [15]:
# performing lemmatization step using WordNetLemmatizer as given
import nltk
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
combined['review_body'] = combined['review_body'].apply(lambda x: lemmatizer.lemmatize(x))

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/trishamandal/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [16]:
#Average length of words after lemmatization by using mean on length of string
avg = combined['review_body'].apply(lambda x: len(x))
avgwordlength4 = avg.mean()
print("Average Review Length before and after preprocessing: {}, {}".format(avgwordlength3, avgwordlength4))

Average Review Length before and after preprocessing: 109.8307, 109.83013


# TF-IDF Feature Extraction

In [17]:
# Using TF-IDF for feature extraction and vectorizing the data
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vec = vectorizer.fit_transform(combined['review_body'])

# Perceptron

In [18]:
# splitting the data into training and test set from train_test_split (80-20 split)
from sklearn.model_selection import train_test_split
labels = combined['star_rating']
xtrain, xtest, ytrain, ytest = train_test_split(vec,combined,test_size=0.2,random_state=2,stratify=labels)
ytest = ytest['star_rating']
ytrain = ytrain['star_rating']

In [19]:
# training the Perceptron model on training data set and then calculating precision score, recall score and accuracy
# score by calling sklearn.metrics functions to calculate the values and then the average of these values

from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.linear_model import Perceptron

clr = Perceptron(random_state=2)
clr = clr.fit(xtrain, ytrain)
prediction = clr.predict(xtest)
recall = recall_score(ytest, prediction, average=None)
f1 = f1_score(ytest, prediction, average=None)
precision = precision_score(ytest, prediction, average=None)
acc = accuracy_score(ytest, prediction)
print("Recall score for every class: ", recall)
print("F1 score for every class: ", f1)
print("Precision score for every class: ", precision)
print("Recall score: ", recall.mean())
print("F1 score: ", f1.mean())
print("Precision score: ", precision.mean())
print("Accuracy score: ", acc.mean())

Recall score for every class:  [0.56    0.209   0.28025 0.4245  0.544  ]
F1 score for every class:  [0.51352591 0.25770654 0.2851329  0.3714317  0.55923927]
Precision score for every class:  [0.47417443 0.33601286 0.29018897 0.3301575  0.57535695]
Recall score:  0.40354999999999996
F1 score:  0.39740726162167367
Precision score:  0.4011781424211677
Accuracy score:  0.40355


# SVM

In [20]:
# training the LinearSVC model on training data set and then calculating precision score, recall score and accuracy
# score by calling sklearn.metrics functions to calculate the values and then the average of these values
from sklearn.svm import LinearSVC
clr2 = LinearSVC(random_state=2)
clr2.fit(xtrain, ytrain)
prediction = clr2.predict(xtest)
recall = recall_score(ytest, prediction, average=None)
f1 = f1_score(ytest, prediction, average=None)
precision = precision_score(ytest, prediction, average=None)
acc = accuracy_score(ytest, prediction)
print("Recall score for every class: ", recall)
print("F1 score for every class: ", f1)
print("Precision score for every class: ", precision)
print("Recall score: ", recall.mean())
print("F1 score: ", f1.mean())
print("Precision score: ", precision.mean())
print("Accuracy score: ", acc.mean())

Recall score for every class:  [0.63675 0.33075 0.33475 0.4015  0.71925]
F1 score for every class:  [0.58754325 0.35060289 0.3630202  0.41828363 0.6593331 ]
Precision score for every class:  [0.54539615 0.37299126 0.39650577 0.43653167 0.60863127]
Recall score:  0.48460000000000003
F1 score:  0.4757566147826324
Precision score:  0.4720112227176513
Accuracy score:  0.4846


# Logistic Regression

In [21]:
# training the LogisticRegression model on training data set and then calculating precision score, recall score and
# accuracy score by calling sklearn.metrics functions to calculate the values and then the average of these values
from sklearn.linear_model import LogisticRegression
clr3 = LogisticRegression(random_state=0).fit(xtrain, ytrain)
prediction = clr3.predict(xtest)
recall = recall_score(ytest, prediction, average=None)
f1 = f1_score(ytest, prediction, average=None)
precision = precision_score(ytest, prediction, average=None)
print("Recall score for every class: ", recall)
print("F1 score for every class: ", f1)
print("Precision score for every class: ", precision)
acc = accuracy_score(ytest, prediction)
print("Recall score: ", recall.mean())
print("F1 score: ", f1.mean())
print("Precision score: ", precision.mean())
print("Accuracy score: ", acc.mean())

Recall score for every class:  [0.61575 0.3935  0.3925  0.443   0.70625]
F1 score for every class:  [0.60286379 0.40405596 0.4005102  0.45558555 0.67110108]
Precision score for every class:  [0.59050587 0.41519388 0.40885417 0.46890712 0.63928491]
Recall score:  0.5102
F1 score:  0.5068233168763573
Precision score:  0.5045491890346222
Accuracy score:  0.5102


# Naive Bayes

In [22]:
# training the MultinomialNB model on training data set and then calculating precision score, recall score and accuracy
# score by calling sklearn.metrics functions to calculate the values and then the average of these values
from sklearn.naive_bayes import MultinomialNB
clr4 = MultinomialNB().fit(xtrain, ytrain)
prediction = clr4.predict(xtest)
recall = recall_score(ytest, prediction, average=None)
f1 = f1_score(ytest, prediction, average=None)
precision = precision_score(ytest, prediction, average=None)
print("Recall score for every class: ", recall)
print("F1 score for every class: ", f1)
print("Precision score for every class: ", precision)
acc = accuracy_score(ytest, prediction)
print("Recall score: ", recall.mean())
print("F1 score: ", f1.mean())
print("Precision score: ", precision.mean())
print("Accuracy score: ", acc.mean())

Recall score for every class:  [0.60025 0.38275 0.39575 0.425   0.67825]
F1 score for every class:  [0.59108813 0.38976578 0.39466467 0.43584156 0.66194949]
Precision score for every class:  [0.58220175 0.39704357 0.39358528 0.44725072 0.64641411]
Recall score:  0.49640000000000006
F1 score:  0.4946619285363082
Precision score:  0.4932990848208808
Accuracy score:  0.4964
